## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
sys.path.append('../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Upernavik,GL,72.7868,-56.1549,1.63,98,100,10.27,overcast clouds
1,1,Buraydah,SA,26.3260,43.9750,68.13,25,36,14.16,scattered clouds
2,2,Arraial Do Cabo,BR,-22.9661,-42.0278,77.67,88,94,13.24,overcast clouds
3,3,Salalah,OM,17.0151,54.0924,71.83,64,1,0.78,clear sky
4,4,Itarema,BR,-2.9248,-39.9167,84.04,65,100,15.23,moderate rain


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Arraial Do Cabo,BR,-22.9661,-42.0278,77.67,88,94,13.24,overcast clouds
3,3,Salalah,OM,17.0151,54.0924,71.83,64,1,0.78,clear sky
4,4,Itarema,BR,-2.9248,-39.9167,84.04,65,100,15.23,moderate rain
9,9,Butaritari,KI,3.0707,172.7902,82.53,73,6,18.14,clear sky
11,11,Cape Town,ZA,-33.9258,18.4232,72.01,53,0,8.99,clear sky


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                230
City                   230
Country                230
Lat                    230
Lng                    230
Max Temp               230
Humidity               230
Cloudiness             230
Wind Speed             230
Current Description    230
dtype: int64

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Arraial Do Cabo,BR,77.67,overcast clouds,-22.9661,-42.0278,
3,Salalah,OM,71.83,clear sky,17.0151,54.0924,
4,Itarema,BR,84.04,moderate rain,-2.9248,-39.9167,
9,Butaritari,KI,82.53,clear sky,3.0707,172.7902,
11,Cape Town,ZA,72.01,clear sky,-33.9258,18.4232,
12,La Romana,DO,81.46,few clouds,18.4273,-68.9728,
13,Rikitea,PF,78.35,broken clouds,-23.1203,-134.9692,
14,Vanimo,PG,77.05,overcast clouds,-2.6741,141.3028,
16,Nang Rong,TH,77.85,scattered clouds,14.6377,102.7914,
18,Pangoa,PE,79.57,overcast clouds,-12.1167,-73.0000,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# Check for null values in hotel DF
hotel_df["Hotel Name"].head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Arraial Do Cabo,BR,77.67,overcast clouds,-22.9661,-42.0278,Pousada Porto Praia
3,Salalah,OM,71.83,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
4,Itarema,BR,84.04,moderate rain,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
9,Butaritari,KI,82.53,clear sky,3.0707,172.7902,Isles Sunset Lodge
11,Cape Town,ZA,72.01,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
12,La Romana,DO,81.46,few clouds,18.4273,-68.9728,Casa de Campo Resort and Villas
13,Rikitea,PF,78.35,broken clouds,-23.1203,-134.9692,People ThankYou
14,Vanimo,PG,77.05,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
16,Nang Rong,TH,77.85,scattered clouds,14.6377,102.7914,Phanomrungpuri Boutique Hotels & Resorts
18,Pangoa,PE,79.57,overcast clouds,-12.1167,-73.0000,


In [45]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]

In [46]:
clean_hotel_df.count()

City                   215
Country                215
Max Temp               215
Current Description    215
Lat                    215
Lng                    215
Hotel Name             215
dtype: int64

In [47]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Arraial Do Cabo,BR,77.67,overcast clouds,-22.9661,-42.0278,Pousada Porto Praia
3,Salalah,OM,71.83,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
4,Itarema,BR,84.04,moderate rain,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
9,Butaritari,KI,82.53,clear sky,3.0707,172.7902,Isles Sunset Lodge
11,Cape Town,ZA,72.01,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
12,La Romana,DO,81.46,few clouds,18.4273,-68.9728,Casa de Campo Resort and Villas
13,Rikitea,PF,78.35,broken clouds,-23.1203,-134.9692,People ThankYou
14,Vanimo,PG,77.05,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
16,Nang Rong,TH,77.85,scattered clouds,14.6377,102.7914,Phanomrungpuri Boutique Hotels & Resorts
20,Dingle,PH,74.05,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY


In [48]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [54]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [55]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))